In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import torch
import numpy as np
import torchvision
import tqdm
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import cv2
from google.colab.patches import cv2_imshow
from torchvision import transforms
from PIL import Image
from matplotlib import pyplot as plt
from torchvision import models
from sklearn.model_selection import train_test_split # for cross-validation
from sklearn.model_selection import StratifiedKFold # for cross-validation


from glob import glob

categories = ['brontosaurus', 'car_side', 'cougar_face', 'dalmatian', 'dollar_bill',
              'dragonfly', 'Faces', 'Faces_easy', 'flamingo', 'headphone',
              'lotus', 'menorah', 'nautilus', 'pagoda', 'soccer_ball',
              'stop_sign', 'sunflower', 'tick', 'windsor_chair', 'yin_yang']
train_img_len = 30
test_img_len = 5
num_of_classes = 20

class ImgDataset(Dataset):
    def __init__(self, method=None):
        self.root = '/content/gdrive/My Drive/'
        self.x_data = []
        self.y_data = []
        self.img_path = []

        if method == 'train':
            self.root = self.root + 'dataset/train'
            for idx, category in enumerate(categories):
                dir_path = self.root + '/' + category + '/'
                self.img_path.append(sorted(glob(dir_path + '*.jpg')))
            

        elif method == 'test':
            self.root = self.root + 'dataset/test'
            for idx, category in enumerate(categories):
                dir_path = self.root + '/' + category + '/'
                self.img_path.append(sorted(glob(dir_path + '*.jpg')))

        if method == 'train':
          length = train_img_len
        elif method == 'test':
          length = test_img_len

        for i in tqdm.tqdm(range(len(self.img_path))):
             for j in range(0,length):
                img = cv2.imread(self.img_path[i][j])
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                self.x_data.append(img)
                self.y_data.append(i)
        

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):        
        #transform1 = torchvision.transforms.ToTensor()
        # Resnet input tensor로 쓰기 위해 resize
        transform1 = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224)),                 
        ])
        new_x_data = transform1(self.x_data[idx])
        return new_x_data, self.y_data[idx]

    def return_whole(self):
        transform1 = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224)),                 
        ])
        new_x_data = []
        for i in range(train_img_len * num_of_classes):
            new_x_data.append(transform1(self.x_data[i]))

        return new_x_data, self.y_data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
net = models.resnet18(pretrained=True)
for p in net.parameters():
  p.requires_grad = False

In [ ]:
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
net.fc = torch.nn.Linear(in_features = 512, out_features=20)

In [ ]:
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
device = 'cuda:0'

class Trainer(object):
    def __init__(self, epochs, batch_size, lr):
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = lr
        self._build_model()

        self.dataset = ImgDataset(method='train')
        self.root = self.dataset.root
        self.dataloader = DataLoader(self.dataset, batch_size=self.batch_size, shuffle=True)

        weight_PATH = self.root.split('/')
        del weight_PATH[-1]
        del weight_PATH[-1]
        
        self.weight_PATH = "/".join(weight_PATH)
        self.weight_PATH = self.weight_PATH + "/"

        # Load of pretrained_weight file
        # weight_PATH = '/content/gdrive/My Drive/
        # self.poseNet.load_state_dict(torch.load(weight_PATH))

        print("Training...")

    def _build_model(self):
        resnet = net
        self.resnet = resnet.to(device)

        print('Finish build model.')

    def return_whole_training_img(self):
        return self.dataset.return_whole()

    def train(self):
        date = '201205'

        loss_func = torch.nn.CrossEntropyLoss()
        #optimizer = torch.optim.Adam(self.resnet.parameters(), lr=self.learning_rate, betas=(0.9, 0.999), eps=1e-08)
        optimizer = torch.optim.SGD(self.resnet.parameters(), lr=self.learning_rate)
        #scheduler = lr_scheduler.ExponentialLR(optimizer, gamma = 0.99)

        self.losses = []
        for epoch in tqdm.tqdm(range(self.epochs + 1)):
            train_loss = 0
            if epoch == self.epochs:
                 torch.save(self.resnet.state_dict(), "".join([self.weight_PATH,'modelweight.pth']))

            for batch_idx, samples in enumerate(self.dataloader):
                optimizer.zero_grad()
                x_train, y_train = samples
                result_resnet = self.resnet(x_train.cuda()).cpu()

                loss = loss_func(result_resnet, y_train)
                train_loss += loss
                loss.backward()
                optimizer.step()

                # Write train result
                if batch_idx % self.batch_size == 0:
                    with open('train_result_' + date + '.txt', 'a') as f:
                        f.write('Epoch {:4d}/{} Batch {}/{}\n'.format(
                            epoch, self.epochs, batch_idx, len(self.dataloader)
                        ))
                    print('Epoch {:4d}/{} Batch {}/{}'.format(
                        epoch, self.epochs, batch_idx, len(self.dataloader)
                    ))
            #scheduler.step()
            self.losses.append(train_loss.item())


        print('Finish training.\n')
    
class Tester(object):
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self._build_model()

        dataset = ImgDataset(method='test')
        self.root = dataset.root
        self.dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False)
        self.datalen = dataset.__len__()
        self.correct = 0

        # load the weight
        weight_PATH = '/content/gdrive/My Drive/modelweight.pth'
        self.resnet.load_state_dict(torch.load(weight_PATH))

        print("Testing...")

    def _build_model(self):
        resnet = net
        self.resnet = resnet.to(device)

        print('Finish build model.')

    def test(self):
        for batch_idx, samples in enumerate(self.dataloader):
            x_test, y_test = samples
            result_resnet = self.resnet(x_test.cuda()).cpu().detach().numpy()
        
            # for each img, call calc_error()
            total = self.datalen


            #soft_func = nn.Softmax(dim=1)
            #output = soft_func(torch.FloatTensor(result_resnet))
            for i in range(self.batch_size):
              res = np.argmax(result_resnet[i])
              if res == y_test[i]:
                self.correct+=1
            
        print('total : ', total)
        print("Accuracy %f" %(self.correct/total))
                
# torch.cuda.isavailable

In [ ]:
class Cross_Validator(object):
    def __init__(self, epochs, batch_size, learning_rate, x_data, y_data):
        self.batch_size = batch_size
        #self.epochs = epochs
        self._build_model()
        self.learning_rate = learning_rate
        self.correct = 0
        self.validation_loss = []
        self.loss = 0
        self.x_data = x_data
        self.y_data = y_data

        self.weight_PATH = '/content/gdrive/My Drive/'
        print("Validating...")

    def _build_model(self):
        resnet = net
        self.resnet = resnet.to(device)
        print('Finish build model.')

    def train(self):
        loss_func = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(self.resnet.parameters(), lr=self.learning_rate)
        self.losses = []

        iteration = len(self.x_data) // self.batch_size              

        idx = 0
        train_loss = 0
        for p in range(iteration):
            optimizer.zero_grad()
            x_mini_batch = np.full((self.batch_size, 3, 224, 224), 0, dtype=np.float32)
            y_mini_batch = np.full((self.batch_size), 0, dtype=np.int_)

            for q in range(self.batch_size):
                x_mini_batch[q] = self.x_data[idx].detach().cpu().numpy()
                y_mini_batch[q] = self.y_data[idx]
                idx+=1

            result_resnet = self.resnet(torch.from_numpy(x_mini_batch).cuda()).cpu()
         
            loss = loss_func(result_resnet, torch.from_numpy(y_mini_batch))

            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
        self.losses.append(train_loss)

        #plt.plot(self.losses)
        #plt.xlabel('epoch')
        #plt.ylabel('Validation Loss While training')
        #plt.show()

        torch.save(self.resnet.state_dict(), "".join([self.weight_PATH,'cross_validation_param.pth']))
        

    def validate(self):
        self.resnet.load_state_dict(torch.load(self.weight_PATH + 'cross_validation_param.pth'))

        loss_func = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(self.resnet.parameters(), lr=self.learning_rate)
       
        validation_loss = 0
        iteration = len(self.x_data) // self.batch_size
        idx = 0

        for p in range(iteration):
            x_mini_batch = np.full((self.batch_size, 3, 224, 224), 0, dtype=np.float32)
            y_mini_batch = np.full((self.batch_size), 0, dtype=np.int_)

            for q in range(self.batch_size):
                x_mini_batch[q] = self.x_data[idx].detach().cpu().numpy()
                y_mini_batch[q] = self.y_data[idx]
                idx+=1

            result_resnet = self.resnet(torch.from_numpy(x_mini_batch).cuda()).cpu()
            loss = loss_func(result_resnet, torch.from_numpy(y_mini_batch))
            validation_loss += loss.item()
        

        print('validation loss in this fold: ', validation_loss)
        self.loss = validation_loss
        

In [ ]:
def main():

    batchSize_candidate = [100,50,20,10] 
    learningRate_candidate = [1e-3,1e-2,1e-1]

    epochs_ = 200
    batchSize_ = 20
    learningRate_ = 1e-2

    trainer = Trainer(epochs_, batchSize_, learningRate_)
    #trainer.train()

    #tester = Tester(batchSize_)
    #tester.test()

'''
    for batchSize in batchSize_candidate:
      for learningRate in learningRate_candidate:
          x_data, y_data = trainer.return_whole_training_img()

          validation_loss = 0
          skf = StratifiedKFold(n_splits=6)
          for train_index, validate_index in skf.split(x_data, y_data): 
              cross_train_x = []
              cross_train_y = []
              cross_validate_x = []
              cross_validate_y = []
 
              for i in train_index:
                  cross_train_x.append(x_data[i])
                  cross_train_y.append(y_data[i])
              for j in validate_index:
                  cross_validate_x.append(x_data[j])
                  cross_validate_y.append(y_data[j])

              cross_trainer = Cross_Validator(1, batchSize, learningRate, cross_train_x, cross_train_y)
              cross_trainer.train()
              cross_validator = Cross_Validator(1, batchSize, learningRate, cross_validate_x, cross_validate_y)
              cross_validator.validate()

              validation_loss += cross_validator.loss

          print('batchsize: %d, learning rate: %f' %(batchSize, learningRate))
          print('Average loss in the 6-fold cross validation: ', validation_loss/ 5)
'''

if __name__ == '__main__':
    main()

 10%|█         | 2/20 [00:00<00:01, 12.27it/s]

Finish build model.


  0%|          | 0/201 [00:00<?, ?it/s]

Training...
Epoch    0/200 Batch 0/30
Epoch    0/200 Batch 20/30


  0%|          | 1/201 [00:01<06:39,  2.00s/it]

...

Epoch  200/200 Batch 0/30
Epoch  200/200 Batch 20/30


 35%|███▌      | 7/20 [00:00<00:00, 59.86it/s]

Finish training.

Finish build model.


100%|██████████| 20/20 [00:00<00:00, 58.68it/s]


Testing...
total :  100
Accuracy 0.850000
